In [ ]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
from IPython.display import clear_output

In [ ]:
# data frame of station locations
station_location_filepath = '/Users/isabellelao/Desktop/NAPS_new/raw_data/StationsNAPS-StationsSNPA.csv'
station_location = pd.read_csv(station_location_filepath, encoding='latin-1')
station_location.columns = station_location.iloc[2]
station_location.drop([0,1,2,3], axis=0, inplace=True)
station_location.reset_index(drop=True, inplace=True)
station_location


In [ ]:
def NAPS_dataextract(filepath,year): 

    if int(year)<2016:
        all_files = glob.glob(filepath+ year + '/*_PM25_' + year +'.xlsx')    
    else:    
        all_files = glob.glob(filepath+ year + '/*_PM25_' + year +'_EN.xlsx')  
   
    all_df = pd.DataFrame()
        
    for file_index in range(len(all_files)):        
        currfile_metadata= pd.ExcelFile(all_files[file_index])
        for ind,e in enumerate(currfile_metadata.sheet_names):
            print (str(ind) + '.', e)        
        sheet_num = input('Insert sheet number(s).\nRemember python indexing starts at 0.\nDo not separate sheet numbers by spaces.\nSeparate by commas if more than 1 sheet.\n')     
        sheet_num = sheet_num.split(',')

        for i in sheet_num:
            curr_sheet = pd.read_excel(all_files[file_index],sheet_name=currfile_metadata.sheet_names[int(i)])
            print(curr_sheet.head(15))
            index = input('Input row number for where the column header is.\nIt probablly is on the 8th row.\n')
            curr_sheet.columns = curr_sheet.iloc[int(index)]
            curr_sheet.drop(range(int(index)+1), axis=0, inplace=True)
            print(curr_sheet.columns)
            columns_wanted = input('Columns wanted separated by commas.\nDo not separate columns by spaces.\nType SKIP if no elements wanted.\n')
            if columns_wanted == 'SKIP':
                break            
            columns_wanted = columns_wanted.split(',')
            clean_df = curr_sheet[columns_wanted].copy()
            columns_toclean = input('Columns to clean separated by commas.\nCleaning means removing all -999 values.\n')
            clean_df.drop(clean_df.loc[clean_df[columns_toclean]==-999].index, inplace=True)
            clean_df = clean_df.reset_index(drop=True)
            all_df = all_df.append(clean_df, ignore_index=True)
            clear_output(wait=True)
            
    return all_df  


In [ ]:
# data frame of WS
all_ws = pd.DataFrame()
years_wanted = ['2016'] # INPUT WHAT YEARS
filepath = '/Users/isabellelao/Desktop/NAPS_new/raw_data/' #INPUT LOCATION OF RAW FILES ORGANIZED BY YEAR
for year in years_wanted:
    df = NAPS_dataextract(filepath,year) 
    all_ws = all_ws.append(df, ignore_index=True) 
       

In [ ]:
all_ws